# Определение уровня навыков игрока на основе анализа результатов бизнес-симуляции

## Московская область. Региональный чемпионат

### 27 сентября — 21 октября 2022

## Задача

### Условие задачи

Разработать и обучить модель машинного обучения на основе метаданных поведения пользователя в игре (лог принятых решений
пользователей в играх) и его итоговой экспертной оценки. В будущем это позволит проставлять оценки участникам симуляции без привлечения экспертов.


### Описание входных значений

    user.csv — таблица, где хранится информация об участнике
    ● user_id — уникальный номер участника
    ● game_id — уникальный номер игры, в которой принимали участие
    ● team_id — уникальный номер команды участника
    user decision.csv — таблица с действиями игрока за время игры
    ● user_id — уникальный номер участника
    ● period_id — номер периода игры (стадии)
    ● decision_id — номер действия, которое совершил участник
    team_point.csv — таблица с баллами команд
    ● team_id — уникальный номер команды
    ● category_id — номер категории, с точки зрении которой оцениваются команды
    ● period_id — номер периода игры (стадии)
    ● score — баллы команды
    ● place — рейтинговый номер команды в конкретном периоде
    decision — таблица с расшифровкой действий участников
    ● decision_id — уникальный номер действия
    ● decision_name — имя действия
    train.csv — файл, содержащий данные игроков для тренировки
    submission.csv — пример файла для отправки, содержит id игроков, для которых требуется предсказать значения


### Метрика
В качестве метрики выступает сумма Recall по четырем навыкам.

## Загрузка библиотек и данных

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import numpy as np
from sklearn.metrics import recall_score, precision_score

%matplotlib inline

In [2]:
df_train = pd.read_csv("train.csv")
df_user = pd.read_csv("user.csv")
df_user_dec = pd.read_csv("user_decision.csv")
df_dec = pd.read_excel("decision.xlsx")
df_point = pd.read_csv("team_point.csv")
df_submission = pd.read_csv("sample_solution.csv")

## Исследование данных

In [3]:
# EDA

def eda(df):
    """ функция выводит основные характеристики датафрейма """
    print(df.head())
    print('\n info()')
    print(df.info())
    print('\n describe()')
    print(df.describe())
    print('\n nunique()')
    print(df.nunique())

def EDA(data):

    shape_info = data.shape

    print(f'\nКоличество строк: {shape_info[0]} \nКоличество столбцов:  {shape_info[1]}\n',
    '\n------------------------------------------------------------------------------------\n')

    print(f'Названия столбцов: {list(data.columns)}\n',
    '\n------------------------------------------------------------------------------------\n')

    print(f'Типы данных:\n')
    print(data.dtypes,
    '\n\n------------------------------------------------------------------------------------\n')

    print('Статистика для численных данных:\n')
    print(data.describe(),
    '\n\n------------------------------------------------------------------------------------\n')
    
    print('Количество уникальных значений для категориальных данных:\n')
    print(data[data.columns[(data.dtypes == object)]].apply(lambda x: x.nunique()),
    '\n\n------------------------------------------------------------------------------------\n')
    
    print('Пропущенные значения:\n')
    print(round(100 * (data.isnull().sum() / data.shape[0]), 2).apply(lambda x: str(x)+'%'),
    '\n\n------------------------------------------------------------------------------------\n')

In [4]:
for i in [df_user, df_user_dec, df_dec, df_point, df_train, df_submission]:
    eda(i)
    print('===============================================')
    print()

   user_id  team_id  game_id
0    10189     1664      235
1    10018     1690      237
2    10580     1394      194
3    10600     1908      262
4    10112     1592      219

 info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 973 entries, 0 to 972
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  973 non-null    int64
 1   team_id  973 non-null    int64
 2   game_id  973 non-null    int64
dtypes: int64(3)
memory usage: 22.9 KB
None

 describe()
            user_id      team_id     game_id
count    973.000000   973.000000  973.000000
mean   10487.000000  1616.429599  224.618705
std      281.025206   179.501299   22.920859
min    10001.000000  1248.000000  182.000000
25%    10244.000000  1474.000000  206.000000
50%    10487.000000  1654.000000  234.000000
75%    10730.000000  1758.000000  243.000000
max    10973.000000  1926.000000  264.000000

 nunique()
user_id    973
team_id    214
game_id     43
dtype: int64



In [5]:
#EDA(df_point)

In [6]:
#df_point[df_point['team_id'] == 1950]

## Подготовка данных

### Предобработка df_dec

In [7]:
df_dec[pd.notnull(df_dec['Unnamed: 2'])]

,decision_id,decision_name,Unnamed: 2
309,310,Implementation of ICS &quot,"\n311"",""Opening of first-aid station"""


In [8]:
df_dec.iloc[309]

decision_id                                        310
decision_name              Implementation of ICS &quot
Unnamed: 2       \n311","Opening of first-aid station"
Name: 309, dtype: object

In [9]:
new_row = {'decision_id':311, 'decision_name':'Opening of first-aid station', 'Unnamed: 2':'NaN'}
df_dec.loc[len(df_dec)] = new_row

In [10]:
new_row = {'decision_id':310, 'decision_name':'Implementation of ICS', 'Unnamed: 2':np.nan}
df_dec.loc[309] = new_row

In [11]:
df_dec = df_dec.drop(columns= ['Unnamed: 2'])
df_dec

,decision_id,decision_name
0,1,Dividend payout in cash
1,2,Dividend payout in stocks
2,3,Zero dividend policy
3,4,The distribution of the residue after the rein...
4,5,Fixed size of dividend payouts sum
...,...,...
433,435,The volume of of transportation - National com...
434,436,The volume of of transportation - Worldwide co...
435,437,Liquidation equipment
436,438,Redeem production room


In [12]:
df_user_dec[df_user_dec['decision_id'] == 311]

,user_id,period,decision_id
306,10948,1,311
325,10929,1,311
513,10208,2,311
961,10583,3,311
992,10609,2,311
...,...,...,...
58239,10735,4,311
58271,10423,3,311
58342,10382,4,311
59002,10665,2,311


In [13]:
lst_dec = df_dec['decision_name'].tolist()
#lst_dec

### df_user_dec . period > sum_user_dec
Количество решений по периодам

In [14]:
df_user_dec

,user_id,period,decision_id
0,10625,1,409
1,10318,3,203
2,10775,4,420
3,10236,2,284
4,10130,3,72
...,...,...,...
60223,10312,3,329
60224,10189,1,33
60225,10346,3,123
60226,10424,2,140


In [15]:
p_count = df_user_dec.groupby(['user_id', 'period']).count()
p_count.columns

Index(['decision_id'], dtype='object')

In [16]:
gb_user_dec = pd.get_dummies(df_user_dec, columns=["period"])
gb_user_dec

,user_id,decision_id,period_1,period_2,period_3,period_4
0,10625,409,1,0,0,0
1,10318,203,0,0,1,0
2,10775,420,0,0,0,1
3,10236,284,0,1,0,0
4,10130,72,0,0,1,0
...,...,...,...,...,...,...
60223,10312,329,0,0,1,0
60224,10189,33,1,0,0,0
60225,10346,123,0,0,1,0
60226,10424,140,0,1,0,0


In [17]:
gb_user_dec[gb_user_dec['user_id'] == 10625]

,user_id,decision_id,period_1,period_2,period_3,period_4
0,10625,409,1,0,0,0
33,10625,67,0,0,1,0
162,10625,148,0,1,0,0
219,10625,176,1,0,0,0
228,10625,146,0,1,0,0
...,...,...,...,...,...,...
58880,10625,408,1,0,0,0
58907,10625,127,0,0,1,0
58936,10625,407,0,0,1,0
59693,10625,109,0,1,0,0


In [18]:
#gb_user_dec.groupby(['user_id', 'period_1', 'period_2', 'period_3', 'period_4']).count()
sum_user_dec = gb_user_dec.groupby(['user_id']).sum()
sum_user_dec

,decision_id,period_1,period_2,period_3,period_4
user_id,,,,,
10001,13677,29,23,39,15
10002,11931,0,9,25,22
10003,34362,21,41,60,84
10004,18534,3,19,41,27
10005,12423,14,34,31,0
...,...,...,...,...,...
10969,3578,0,1,5,8
10970,20701,11,22,32,0
10971,19457,22,26,12,25


In [19]:
# sum_user_dec['period_sum'] = sum_user_dec['period_1'] + sum_user_dec['period_2'] + sum_user_dec['period_3'] + sum_user_dec['period_4']
# sum_user_dec['period_12'] = sum_user_dec['period_1'] + sum_user_dec['period_2']
# sum_user_dec['period_23'] = sum_user_dec['period_2'] + sum_user_dec['period_3']
# sum_user_dec['period_34'] = sum_user_dec['period_3'] + sum_user_dec['period_4']
# sum_user_dec['period_41'] = sum_user_dec['period_4'] + sum_user_dec['period_1']
# sum_user_dec

In [20]:
#sum_user_dec

### df_user_dec . decision_id > sumdec_user_dec
Принимаемые решения

In [21]:
d_count = df_user_dec.groupby(['user_id', 'decision_id']).count()
d_count

period
user_id decision_id        
10001   1                 1
        4                 1
        6                 1
        11                2
        12                2
...                     ...
10973   304               1
        306               1
        344               1
        386               1
        427               1

[44663 rows x 1 columns]

In [22]:
gbdec_user_dec = pd.get_dummies(df_user_dec, columns=["decision_id"])
gbdec_user_dec

,user_id,period,decision_id_1,decision_id_3,decision_id_4,decision_id_5,decision_id_6,decision_id_7,decision_id_8,decision_id_9,...,decision_id_427,decision_id_428,decision_id_429,decision_id_430,decision_id_432,decision_id_433,decision_id_434,decision_id_435,decision_id_436,decision_id_437
0,10625,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10318,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10775,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10236,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10130,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60223,10312,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60224,10189,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60225,10346,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60226,10424,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
sumdec_user_dec = gbdec_user_dec.groupby(['user_id']).sum()
sumdec_user_dec = sumdec_user_dec * 10 
sumdec_user_dec

,period,decision_id_1,decision_id_3,decision_id_4,decision_id_5,decision_id_6,decision_id_7,decision_id_8,decision_id_9,decision_id_10,...,decision_id_427,decision_id_428,decision_id_429,decision_id_430,decision_id_432,decision_id_433,decision_id_434,decision_id_435,decision_id_436,decision_id_437
user_id,,,,,,,,,,,,,,,,,,,,,
10001,2520,10,0,10,0,10,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10002,1810,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10003,6190,0,0,0,0,0,0,0,0,0,...,0,0,10,10,0,0,0,0,0,0
10004,2720,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,10,10,0,0
10005,1750,0,0,0,0,0,0,0,0,10,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,490,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10970,1510,0,0,0,0,0,0,0,0,0,...,30,10,10,10,20,10,10,0,30,0
10971,2100,0,0,0,0,0,0,0,0,0,...,10,0,20,0,0,0,0,0,0,0


In [24]:
sumdec_user_dec_x = sumdec_user_dec.copy()
sumdec_user_dec_x = sumdec_user_dec_x * 10 
sumdec_user_dec_x

,period,decision_id_1,decision_id_3,decision_id_4,decision_id_5,decision_id_6,decision_id_7,decision_id_8,decision_id_9,decision_id_10,...,decision_id_427,decision_id_428,decision_id_429,decision_id_430,decision_id_432,decision_id_433,decision_id_434,decision_id_435,decision_id_436,decision_id_437
user_id,,,,,,,,,,,,,,,,,,,,,
10001,25200,100,0,100,0,100,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10002,18100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10003,61900,0,0,0,0,0,0,0,0,0,...,0,0,100,100,0,0,0,0,0,0
10004,27200,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,100,100,0,0
10005,17500,0,0,0,0,0,0,0,0,100,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,4900,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10970,15100,0,0,0,0,0,0,0,0,0,...,44,100,100,100,200,100,100,0,44,0
10971,21000,0,0,0,0,0,0,0,0,0,...,100,0,200,0,0,0,0,0,0,0


### train
Формирование таблицы для обучения

In [25]:
df_train

,id,Analytical thinking,Systemic thinking,Adaptability,Focus
0,10884,4.0,4.0,4.0,4.0
1,10106,5.0,4.0,5.0,5.0
2,10438,6.0,5.0,5.0,5.0
3,10130,5.0,4.0,5.0,5.0
4,10667,4.0,5.0,5.0,5.0
...,...,...,...,...,...
675,10563,5.0,4.0,5.0,4.0
676,10386,5.0,4.0,5.0,5.0
677,10675,5.0,4.0,5.0,4.0
678,10366,4.0,4.0,4.0,4.0


In [26]:
sum_user_dec

,decision_id,period_1,period_2,period_3,period_4
user_id,,,,,
10001,13677,29,23,39,15
10002,11931,0,9,25,22
10003,34362,21,41,60,84
10004,18534,3,19,41,27
10005,12423,14,34,31,0
...,...,...,...,...,...
10969,3578,0,1,5,8
10970,20701,11,22,32,0
10971,19457,22,26,12,25


In [27]:
# Объединение df
train = pd.merge(df_train, sum_user_dec, how='left', left_on='id', right_on='user_id')
train = train.drop(columns= ['decision_id'])
train.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4
0,10884,4.0,4.0,4.0,4.0,7,14,28,10
1,10106,5.0,4.0,5.0,5.0,21,21,12,8
2,10438,6.0,5.0,5.0,5.0,62,45,63,38
3,10130,5.0,4.0,5.0,5.0,26,28,34,66
4,10667,4.0,5.0,5.0,5.0,58,70,194,35
5,10908,4.0,4.0,5.0,5.0,0,4,10,7
6,10378,4.0,5.0,5.0,4.0,3,1,0,0
7,10220,5.0,4.0,5.0,5.0,1,0,2,1
8,10612,5.0,4.0,4.0,5.0,26,22,12,15
9,10776,4.0,4.0,5.0,5.0,16,13,37,36


In [28]:
sumdec_user_dec

,period,decision_id_1,decision_id_3,decision_id_4,decision_id_5,decision_id_6,decision_id_7,decision_id_8,decision_id_9,decision_id_10,...,decision_id_427,decision_id_428,decision_id_429,decision_id_430,decision_id_432,decision_id_433,decision_id_434,decision_id_435,decision_id_436,decision_id_437
user_id,,,,,,,,,,,,,,,,,,,,,
10001,2520,10,0,10,0,10,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10002,1810,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10003,6190,0,0,0,0,0,0,0,0,0,...,0,0,10,10,0,0,0,0,0,0
10004,2720,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,10,10,0,0
10005,1750,0,0,0,0,0,0,0,0,10,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10969,490,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10970,1510,0,0,0,0,0,0,0,0,0,...,30,10,10,10,20,10,10,0,30,0
10971,2100,0,0,0,0,0,0,0,0,0,...,10,0,20,0,0,0,0,0,0,0


In [29]:
sumdec_user_dec = sumdec_user_dec.drop(columns= ['period'])
train = pd.merge(train, sumdec_user_dec, how='left', left_on='id', right_on='user_id')
train.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,decision_id_1,...,decision_id_427,decision_id_428,decision_id_429,decision_id_430,decision_id_432,decision_id_433,decision_id_434,decision_id_435,decision_id_436,decision_id_437
0,10884,4.0,4.0,4.0,4.0,7,14,28,10,0,...,20,0,0,0,0,0,0,0,0,0
1,10106,5.0,4.0,5.0,5.0,21,21,12,8,0,...,0,0,0,0,0,0,0,0,0,0
2,10438,6.0,5.0,5.0,5.0,62,45,63,38,0,...,20,0,0,0,0,0,0,0,0,0
3,10130,5.0,4.0,5.0,5.0,26,28,34,66,0,...,0,10,0,0,0,0,0,0,20,0
4,10667,4.0,5.0,5.0,5.0,58,70,194,35,10,...,20,0,10,0,10,10,10,10,10,0
5,10908,4.0,4.0,5.0,5.0,0,4,10,7,0,...,0,0,0,0,0,0,0,0,0,0
6,10378,4.0,5.0,5.0,4.0,3,1,0,0,10,...,10,10,0,0,0,0,0,0,0,0
7,10220,5.0,4.0,5.0,5.0,1,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
8,10612,5.0,4.0,4.0,5.0,26,22,12,15,0,...,0,10,10,20,0,0,0,0,0,0
9,10776,4.0,4.0,5.0,5.0,16,13,37,36,0,...,0,0,0,0,0,0,0,0,0,0


### df_user
ID команды и ID игры

In [30]:
df_user

,user_id,team_id,game_id
0,10189,1664,235
1,10018,1690,237
2,10580,1394,194
3,10600,1908,262
4,10112,1592,219
...,...,...,...
968,10205,1826,249
969,10325,1628,231
970,10527,1730,241
971,10371,1704,238


In [31]:
# Объединение df
train = pd.merge(train, df_user, how='left', left_on='id', right_on='user_id')
train = train.drop(columns= ['user_id'])
train.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,decision_id_1,...,decision_id_429,decision_id_430,decision_id_432,decision_id_433,decision_id_434,decision_id_435,decision_id_436,decision_id_437,team_id,game_id
0,10884,4.0,4.0,4.0,4.0,7,14,28,10,0,...,0,0,0,0,0,0,0,0,1440,202
1,10106,5.0,4.0,5.0,5.0,21,21,12,8,0,...,0,0,0,0,0,0,0,0,1536,211
2,10438,6.0,5.0,5.0,5.0,62,45,63,38,0,...,0,0,0,0,0,0,0,0,1742,242
3,10130,5.0,4.0,5.0,5.0,26,28,34,66,0,...,0,0,0,0,0,0,20,0,1272,184
4,10667,4.0,5.0,5.0,5.0,58,70,194,35,10,...,10,0,10,10,10,10,10,0,1382,193
5,10908,4.0,4.0,5.0,5.0,0,4,10,7,0,...,0,0,0,0,0,0,0,0,1750,243
6,10378,4.0,5.0,5.0,4.0,3,1,0,0,10,...,0,0,0,0,0,0,0,0,1468,205
7,10220,5.0,4.0,5.0,5.0,1,0,2,1,0,...,0,0,0,0,0,0,0,0,1668,235
8,10612,5.0,4.0,4.0,5.0,26,22,12,15,0,...,10,20,0,0,0,0,0,0,1346,190
9,10776,4.0,4.0,5.0,5.0,16,13,37,36,0,...,0,0,0,0,0,0,0,0,1688,237


### df_point . period > point_per
Количество очков по периодам игры

In [32]:
df_point = pd.read_csv("team_point.csv")
df_point.head(3)

,team_id,category_id,period,score,place
0,1948,2,0,16,2
1,1934,3,4,45,6
2,1894,4,8,24,4


In [33]:
df_point

,team_id,category_id,period,score,place
0,1948,2,0,16,2
1,1934,3,4,45,6
2,1894,4,8,24,4
3,1688,4,0,27,4
4,1592,4,2,21,5
...,...,...,...,...,...
10981,1312,4,1,32,4
10982,1974,1,0,4,6
10983,1822,1,4,31,1
10984,1390,2,0,16,2


In [34]:
#df_point[df_point['team_id'] == 1414].sort_values(['category_id','period'], ascending=True)

In [35]:
# берем только 4 первых периода
df_point_4 = df_point[df_point['period'] < 4]
df_point_4 

,team_id,category_id,period,score,place
0,1948,2,0,16,2
3,1688,4,0,27,4
4,1592,4,2,21,5
5,1344,5,0,8,6
6,1982,6,1,146,3
...,...,...,...,...,...
10980,1236,5,1,15,1
10981,1312,4,1,32,4
10982,1974,1,0,4,6
10984,1390,2,0,16,2


In [36]:
#!!! 
point_per = df_point_4.groupby(["team_id", "period"]).agg({
  "score": "sum"
}).reset_index()
point_per

,team_id,period,score
0,381,0,316
1,381,1,264
2,381,2,220
3,381,3,26
4,382,0,316
...,...,...,...
1415,1988,3,54
1416,1990,0,316
1417,1990,1,270
1418,1990,2,332


In [37]:
point_per = point_per.rename(columns={'period': 'period_team'})
point_per

,team_id,period_team,score
0,381,0,316
1,381,1,264
2,381,2,220
3,381,3,26
4,382,0,316
...,...,...,...
1415,1988,3,54
1416,1990,0,316
1417,1990,1,270
1418,1990,2,332


In [38]:
point_per = pd.get_dummies(point_per, columns=["period_team"])
point_per

,team_id,score,period_team_0,period_team_1,period_team_2,period_team_3
0,381,316,1,0,0,0
1,381,264,0,1,0,0
2,381,220,0,0,1,0
3,381,26,0,0,0,1
4,382,316,1,0,0,0
...,...,...,...,...,...,...
1415,1988,54,0,0,0,1
1416,1990,316,1,0,0,0
1417,1990,270,0,1,0,0
1418,1990,332,0,0,1,0


In [39]:
point_per.period_team_0 = point_per.period_team_0 * point_per.score
point_per.period_team_1 = point_per.period_team_1 * point_per.score
point_per.period_team_2 = point_per.period_team_2 * point_per.score
point_per.period_team_3 = point_per.period_team_3 * point_per.score
point_per

,team_id,score,period_team_0,period_team_1,period_team_2,period_team_3
0,381,316,316,0,0,0
1,381,264,0,264,0,0
2,381,220,0,0,220,0
3,381,26,0,0,0,26
4,382,316,316,0,0,0
...,...,...,...,...,...,...
1415,1988,54,0,0,0,54
1416,1990,316,316,0,0,0
1417,1990,270,0,270,0,0
1418,1990,332,0,0,332,0


In [40]:
point_per = point_per.groupby(["team_id"]).agg({
  "period_team_0": "sum", "period_team_1": "sum", "period_team_2": "sum", "period_team_3": "sum"
}).reset_index()
point_per

,team_id,period_team_0,period_team_1,period_team_2,period_team_3
0,381,316,264,220,26
1,382,316,264,332,178
2,383,316,230,378,392
3,384,316,404,246,154
4,1232,316,232,74,16
...,...,...,...,...,...
350,1982,316,292,464,76
351,1984,316,342,580,248
352,1986,316,262,176,170
353,1988,316,210,260,54


In [41]:
point_per.describe()

,team_id,period_team_0,period_team_1,period_team_2,period_team_3
count,355.000000,355.0,355.000000,355.000000,355.000000
mean,1602.957746,316.0,300.619718,324.507042,303.250704
std,256.207699,0.0,111.974670,144.582125,180.700307
min,381.000000,316.0,-36.000000,-30.000000,-36.000000
25%,1421.000000,316.0,226.000000,237.000000,154.000000
50%,1620.000000,316.0,282.000000,316.000000,290.000000
75%,1809.000000,316.0,351.000000,415.000000,426.000000
max,1990.000000,316.0,600.000000,680.000000,794.000000


### df_point . category_id > point_cat
Количество очков по категориям

In [42]:
df_point['place'].max()

6

In [43]:
df_point['place'] = df_point['place'].max() - df_point['place'] + 1
df_point.head(3)

,team_id,category_id,period,score,place
0,1948,2,0,16,5
1,1934,3,4,45,1
2,1894,4,8,24,3


In [44]:
#df_point[df_point['team_id'] == 1414].sort_values(['category_id','period'], ascending=True)

In [45]:
point_sum2 = df_point.groupby(["team_id", "category_id"]).agg({
  "score": "sum", "period": "count"
}).reset_index()
point_sum2

,team_id,category_id,score,period
0,381,1,-31,5
1,381,2,-49,5
2,381,3,320,5
3,381,4,116,5
4,381,5,112,5
...,...,...,...,...
2125,1990,2,38,5
2126,1990,3,339,5
2127,1990,4,133,5
2128,1990,5,156,5


In [46]:
#!!!
point_sum = df_point.groupby(["team_id", "category_id"]).agg({
  "score": "sum"
}).reset_index()
point_sum

,team_id,category_id,score
0,381,1,-31
1,381,2,-49
2,381,3,320
3,381,4,116
4,381,5,112
...,...,...,...
2125,1990,2,38
2126,1990,3,339
2127,1990,4,133
2128,1990,5,156


In [47]:
point_cat = pd.get_dummies(point_sum, columns=["category_id"])
point_cat

,team_id,score,category_id_1,category_id_2,category_id_3,category_id_4,category_id_5,category_id_6
0,381,-31,1,0,0,0,0,0
1,381,-49,0,1,0,0,0,0
2,381,320,0,0,1,0,0,0
3,381,116,0,0,0,1,0,0
4,381,112,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
2125,1990,38,0,1,0,0,0,0
2126,1990,339,0,0,1,0,0,0
2127,1990,133,0,0,0,1,0,0
2128,1990,156,0,0,0,0,1,0


In [48]:
point_cat.category_id_1 = point_cat.category_id_1 * point_cat.score
point_cat.category_id_2 = point_cat.category_id_2 * point_cat.score
point_cat.category_id_3 = point_cat.category_id_3 * point_cat.score
point_cat.category_id_4 = point_cat.category_id_4 * point_cat.score
point_cat.category_id_5 = point_cat.category_id_5 * point_cat.score
point_cat.category_id_6 = point_cat.category_id_6 * point_cat.score
point_cat

,team_id,score,category_id_1,category_id_2,category_id_3,category_id_4,category_id_5,category_id_6
0,381,-31,-31,0,0,0,0,0
1,381,-49,0,-49,0,0,0,0
2,381,320,0,0,320,0,0,0
3,381,116,0,0,0,116,0,0
4,381,112,0,0,0,0,112,0
...,...,...,...,...,...,...,...,...
2125,1990,38,0,38,0,0,0,0
2126,1990,339,0,0,339,0,0,0
2127,1990,133,0,0,0,133,0,0
2128,1990,156,0,0,0,0,156,0


In [49]:
point_cat2 = point_cat.groupby(["team_id"]).agg({
  "category_id_1": "sum", "category_id_2": "sum", "category_id_3": "sum", "category_id_4": "sum", "category_id_5": "sum", "category_id_6": "sum"
}).reset_index()
point_cat2

,team_id,category_id_1,category_id_2,category_id_3,category_id_4,category_id_5,category_id_6
0,381,-31,-49,320,116,112,468
1,382,-3,28,347,141,115,628
2,383,28,138,363,155,93,777
3,384,-8,5,371,128,127,623
4,1232,-75,-98,319,98,145,389
...,...,...,...,...,...,...,...
350,1982,-1,-21,417,132,107,634
351,1984,91,173,404,156,116,940
352,1986,-10,-20,383,150,114,617
353,1988,-33,-49,332,128,111,489


In [50]:
train

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,decision_id_1,...,decision_id_429,decision_id_430,decision_id_432,decision_id_433,decision_id_434,decision_id_435,decision_id_436,decision_id_437,team_id,game_id
0,10884,4.0,4.0,4.0,4.0,7,14,28,10,0,...,0,0,0,0,0,0,0,0,1440,202
1,10106,5.0,4.0,5.0,5.0,21,21,12,8,0,...,0,0,0,0,0,0,0,0,1536,211
2,10438,6.0,5.0,5.0,5.0,62,45,63,38,0,...,0,0,0,0,0,0,0,0,1742,242
3,10130,5.0,4.0,5.0,5.0,26,28,34,66,0,...,0,0,0,0,0,0,20,0,1272,184
4,10667,4.0,5.0,5.0,5.0,58,70,194,35,10,...,10,0,10,10,10,10,10,0,1382,193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,10563,5.0,4.0,5.0,4.0,10,2,14,4,0,...,0,0,0,0,0,0,0,0,1274,184
676,10386,5.0,4.0,5.0,5.0,1,14,24,1,0,...,0,0,0,0,0,0,0,0,1526,210
677,10675,5.0,4.0,5.0,4.0,4,17,23,12,0,...,30,0,10,20,40,30,0,0,1670,235
678,10366,4.0,4.0,4.0,4.0,10,44,45,38,10,...,0,0,0,0,0,0,0,0,1808,248


In [51]:
# Объединение с point_cat2
train = pd.merge(train, point_cat2, how='left', left_on='team_id', right_on='team_id')
#train = train.drop(columns= ['user_id'])
train.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,decision_id_1,...,decision_id_436,decision_id_437,team_id,game_id,category_id_1,category_id_2,category_id_3,category_id_4,category_id_5,category_id_6
0,10884,4.0,4.0,4.0,4.0,7,14,28,10,0,...,0,0,1440,202,-5,-25,391,128,105,594
1,10106,5.0,4.0,5.0,5.0,21,21,12,8,0,...,0,0,1536,211,54,120,467,160,113,914
2,10438,6.0,5.0,5.0,5.0,62,45,63,38,0,...,0,0,1742,242,55,60,383,158,146,802
3,10130,5.0,4.0,5.0,5.0,26,28,34,66,0,...,20,0,1272,184,31,24,366,139,107,667
4,10667,4.0,5.0,5.0,5.0,58,70,194,35,10,...,10,0,1382,193,71,70,416,166,155,878
5,10908,4.0,4.0,5.0,5.0,0,4,10,7,0,...,0,0,1750,243,26,68,477,150,70,791
6,10378,4.0,5.0,5.0,4.0,3,1,0,0,10,...,0,0,1468,205,81,156,498,167,82,984
7,10220,5.0,4.0,5.0,5.0,1,0,2,1,0,...,0,0,1668,235,25,80,440,139,65,749
8,10612,5.0,4.0,4.0,5.0,26,22,12,15,0,...,0,0,1346,190,75,156,504,165,70,970
9,10776,4.0,4.0,5.0,5.0,16,13,37,36,0,...,0,0,1688,237,0,-5,334,119,73,521


In [52]:
# Объединение с point_per
train = pd.merge(train, point_per, how='left', left_on='team_id', right_on='team_id')
#train = train.drop(columns= ['user_id'])
train.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,decision_id_1,...,category_id_1,category_id_2,category_id_3,category_id_4,category_id_5,category_id_6,period_team_0,period_team_1,period_team_2,period_team_3
0,10884,4.0,4.0,4.0,4.0,7,14,28,10,0,...,-5,-25,391,128,105,594,316,284,166,120
1,10106,5.0,4.0,5.0,5.0,21,21,12,8,0,...,54,120,467,160,113,914,316,338,468,398
2,10438,6.0,5.0,5.0,5.0,62,45,63,38,0,...,55,60,383,158,146,802,316,184,598,340
3,10130,5.0,4.0,5.0,5.0,26,28,34,66,0,...,31,24,366,139,107,667,316,164,160,300
4,10667,4.0,5.0,5.0,5.0,58,70,194,35,10,...,71,70,416,166,155,878,316,558,576,150
5,10908,4.0,4.0,5.0,5.0,0,4,10,7,0,...,26,68,477,150,70,791,316,320,370,230
6,10378,4.0,5.0,5.0,4.0,3,1,0,0,10,...,81,156,498,167,82,984,316,408,532,360
7,10220,5.0,4.0,5.0,5.0,1,0,2,1,0,...,25,80,440,139,65,749,316,228,324,340
8,10612,5.0,4.0,4.0,5.0,26,22,12,15,0,...,75,156,504,165,70,970,316,420,478,432
9,10776,4.0,4.0,5.0,5.0,16,13,37,36,0,...,0,-5,334,119,73,521,316,216,258,128


### test
Формирование тестовой таблицы

In [53]:
# Объединение df

In [54]:
test = df_submission.copy()

In [55]:
test = pd.merge(test, sum_user_dec, how='left', left_on='id', right_on='user_id')
test = test.drop(columns= ['decision_id'])
test.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4
0,10199,0,0,0,0,2,2,3,2
1,10539,0,0,0,0,1,9,15,17
2,10174,0,0,0,0,0,28,31,20
3,10465,0,0,0,0,15,18,32,18
4,10066,0,0,0,0,8,23,16,6
5,10425,0,0,0,0,2,0,7,4
6,10398,0,0,0,0,2,44,40,30
7,10346,0,0,0,0,16,19,24,4
8,10281,0,0,0,0,2,6,12,9
9,10158,0,0,0,0,3,6,1,7


In [56]:
#sumdec_user_dec = sumdec_user_dec.drop(columns= ['period'])
test = pd.merge(test, sumdec_user_dec, how='left', left_on='id', right_on='user_id')
test.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,decision_id_1,...,decision_id_427,decision_id_428,decision_id_429,decision_id_430,decision_id_432,decision_id_433,decision_id_434,decision_id_435,decision_id_436,decision_id_437
0,10199,0,0,0,0,2,2,3,2,0,...,0,0,0,0,0,0,0,0,0,0
1,10539,0,0,0,0,1,9,15,17,10,...,10,0,0,0,0,0,0,0,0,0
2,10174,0,0,0,0,0,28,31,20,0,...,30,10,30,0,0,0,0,0,0,0
3,10465,0,0,0,0,15,18,32,18,0,...,0,0,0,0,0,0,0,0,0,0
4,10066,0,0,0,0,8,23,16,6,0,...,0,0,0,0,0,0,0,0,0,0
5,10425,0,0,0,0,2,0,7,4,0,...,0,0,0,0,0,0,0,0,0,0
6,10398,0,0,0,0,2,44,40,30,0,...,0,0,0,0,0,0,0,0,0,0
7,10346,0,0,0,0,16,19,24,4,0,...,0,0,0,0,0,0,0,0,0,0
8,10281,0,0,0,0,2,6,12,9,0,...,0,0,0,0,0,0,0,0,0,0
9,10158,0,0,0,0,3,6,1,7,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
test = pd.merge(test, df_user, how='left', left_on='id', right_on='user_id')
test = test.drop(columns= ['user_id'])
test.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,decision_id_1,...,decision_id_429,decision_id_430,decision_id_432,decision_id_433,decision_id_434,decision_id_435,decision_id_436,decision_id_437,team_id,game_id
0,10199,0,0,0,0,2,2,3,2,0,...,0,0,0,0,0,0,0,0,1912,262
1,10539,0,0,0,0,1,9,15,17,10,...,0,0,0,0,0,0,0,0,1458,204
2,10174,0,0,0,0,0,28,31,20,0,...,30,0,0,0,0,0,0,0,1348,190
3,10465,0,0,0,0,15,18,32,18,0,...,0,0,0,0,0,0,0,0,1760,244
4,10066,0,0,0,0,8,23,16,6,0,...,0,0,0,0,0,0,0,0,1260,183
5,10425,0,0,0,0,2,0,7,4,0,...,0,0,0,0,0,0,0,0,1724,241
6,10398,0,0,0,0,2,44,40,30,0,...,0,0,0,0,0,0,0,0,1908,262
7,10346,0,0,0,0,16,19,24,4,0,...,0,0,0,0,0,0,0,0,1656,234
8,10281,0,0,0,0,2,6,12,9,0,...,0,0,0,0,0,0,0,0,1660,234
9,10158,0,0,0,0,3,6,1,7,0,...,0,0,0,0,0,0,0,0,1708,238


In [58]:
# Объединение с point_cat2
test = pd.merge(test, point_cat2, how='left', left_on='team_id', right_on='team_id')
#train = train.drop(columns= ['user_id'])
test.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,decision_id_1,...,decision_id_436,decision_id_437,team_id,game_id,category_id_1,category_id_2,category_id_3,category_id_4,category_id_5,category_id_6
0,10199,0,0,0,0,2,2,3,2,0,...,0,0,1912,262,11,56,387,151,107,712
1,10539,0,0,0,0,1,9,15,17,10,...,0,0,1458,204,-18,-3,356,118,131,584
2,10174,0,0,0,0,0,28,31,20,0,...,0,0,1348,190,124,206,481,201,122,1134
3,10465,0,0,0,0,15,18,32,18,0,...,0,0,1760,244,-23,-19,422,132,114,626
4,10066,0,0,0,0,8,23,16,6,0,...,0,0,1260,183,-48,-80,314,115,94,395
5,10425,0,0,0,0,2,0,7,4,0,...,0,0,1724,241,59,145,532,155,71,962
6,10398,0,0,0,0,2,44,40,30,0,...,0,0,1908,262,22,134,353,178,94,781
7,10346,0,0,0,0,16,19,24,4,0,...,0,0,1656,234,48,119,367,144,142,820
8,10281,0,0,0,0,2,6,12,9,0,...,0,0,1660,234,-3,-30,368,129,73,537
9,10158,0,0,0,0,3,6,1,7,0,...,0,0,1708,238,-15,-45,315,108,75,438


In [59]:
# Объединение с point_per
test = pd.merge(test, point_per, how='left', left_on='team_id', right_on='team_id')
test.head(10)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,decision_id_1,...,category_id_1,category_id_2,category_id_3,category_id_4,category_id_5,category_id_6,period_team_0,period_team_1,period_team_2,period_team_3
0,10199,0,0,0,0,2,2,3,2,0,...,11,56,387,151,107,712,316,404,330,182
1,10539,0,0,0,0,1,9,15,17,10,...,-18,-3,356,118,131,584,316,214,362,82
2,10174,0,0,0,0,0,28,31,20,0,...,124,206,481,201,122,1134,316,554,636,410
3,10465,0,0,0,0,15,18,32,18,0,...,-23,-19,422,132,114,626,316,322,288,160
4,10066,0,0,0,0,8,23,16,6,0,...,-48,-80,314,115,94,395,316,266,24,32
5,10425,0,0,0,0,2,0,7,4,0,...,59,145,532,155,71,962,316,436,410,342
6,10398,0,0,0,0,2,44,40,30,0,...,22,134,353,178,94,781,316,426,54,468
7,10346,0,0,0,0,16,19,24,4,0,...,48,119,367,144,142,820,316,304,254,236
8,10281,0,0,0,0,2,6,12,9,0,...,-3,-30,368,129,73,537,316,128,236,152
9,10158,0,0,0,0,3,6,1,7,0,...,-15,-45,315,108,75,438,316,188,10,62


## Выделим выборки

In [60]:
train

,id,Analytical thinking,Systemic thinking,Adaptability,Focus,period_1,period_2,period_3,period_4,decision_id_1,...,category_id_1,category_id_2,category_id_3,category_id_4,category_id_5,category_id_6,period_team_0,period_team_1,period_team_2,period_team_3
0,10884,4.0,4.0,4.0,4.0,7,14,28,10,0,...,-5,-25,391,128,105,594,316,284,166,120
1,10106,5.0,4.0,5.0,5.0,21,21,12,8,0,...,54,120,467,160,113,914,316,338,468,398
2,10438,6.0,5.0,5.0,5.0,62,45,63,38,0,...,55,60,383,158,146,802,316,184,598,340
3,10130,5.0,4.0,5.0,5.0,26,28,34,66,0,...,31,24,366,139,107,667,316,164,160,300
4,10667,4.0,5.0,5.0,5.0,58,70,194,35,10,...,71,70,416,166,155,878,316,558,576,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,10563,5.0,4.0,5.0,4.0,10,2,14,4,0,...,-4,-27,349,127,91,536,316,226,174,102
676,10386,5.0,4.0,5.0,5.0,1,14,24,1,0,...,37,108,471,154,62,832,316,364,282,396
677,10675,5.0,4.0,5.0,4.0,4,17,23,12,0,...,31,86,399,136,81,733,316,338,244,230
678,10366,4.0,4.0,4.0,4.0,10,44,45,38,10,...,45,108,425,164,100,842,316,426,410,392


In [61]:
col_name = ['Analytical thinking', 'Systemic thinking', 'Adaptability', 'Focus']
X = train.drop(columns=col_name)
y = train[col_name]

In [62]:
X

,id,period_1,period_2,period_3,period_4,decision_id_1,decision_id_3,decision_id_4,decision_id_5,decision_id_6,...,category_id_1,category_id_2,category_id_3,category_id_4,category_id_5,category_id_6,period_team_0,period_team_1,period_team_2,period_team_3
0,10884,7,14,28,10,0,0,0,0,0,...,-5,-25,391,128,105,594,316,284,166,120
1,10106,21,21,12,8,0,0,0,0,0,...,54,120,467,160,113,914,316,338,468,398
2,10438,62,45,63,38,0,0,0,0,0,...,55,60,383,158,146,802,316,184,598,340
3,10130,26,28,34,66,0,0,0,0,0,...,31,24,366,139,107,667,316,164,160,300
4,10667,58,70,194,35,10,0,0,0,10,...,71,70,416,166,155,878,316,558,576,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,10563,10,2,14,4,0,0,0,0,0,...,-4,-27,349,127,91,536,316,226,174,102
676,10386,1,14,24,1,0,0,0,0,0,...,37,108,471,154,62,832,316,364,282,396
677,10675,4,17,23,12,0,0,0,0,0,...,31,86,399,136,81,733,316,338,244,230
678,10366,10,44,45,38,10,0,0,0,0,...,45,108,425,164,100,842,316,426,410,392


In [63]:
y

,Analytical thinking,Systemic thinking,Adaptability,Focus
0,4.0,4.0,4.0,4.0
1,5.0,4.0,5.0,5.0
2,6.0,5.0,5.0,5.0
3,5.0,4.0,5.0,5.0
4,4.0,5.0,5.0,5.0
...,...,...,...,...
675,5.0,4.0,5.0,4.0
676,5.0,4.0,5.0,5.0
677,5.0,4.0,5.0,4.0
678,4.0,4.0,4.0,4.0


In [64]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## CatBoostClassifier

### Обучение модели

In [65]:
y.head(1)

,Analytical thinking,Systemic thinking,Adaptability,Focus
0,4.0,4.0,4.0,4.0


In [66]:
y1_train = y_train['Analytical thinking']
y2_train = y_train['Systemic thinking']
y3_train = y_train['Adaptability']
y4_train = y_train['Focus']
y1_train.head(3)

678    4.0
448    5.0
673    4.0
Name: Analytical thinking, dtype: float64

In [67]:
#Создание объекта данных Pool, плюсы: возможность указать какие признаки являются категориальными
from catboost import CatBoostRegressor, CatBoostClassifier, Pool, cv
from sklearn.metrics import r2_score
pool_train1 = Pool(X_train, y1_train)
pool_train2 = Pool(X_train, y2_train)
pool_train3 = Pool(X_train, y3_train)
pool_train4 = Pool(X_train, y4_train)
pool_val = Pool(X_val)

In [68]:
%%time
cb_model1 = CatBoostClassifier(depth = 6, iterations = 1000)
cb_model2 = CatBoostClassifier(depth = 6, iterations = 1000)
cb_model3 = CatBoostClassifier(depth = 6, iterations = 1000)
cb_model4 = CatBoostClassifier(depth = 6, iterations = 1000)

CPU times: total: 0 ns
Wall time: 1 ms


In [69]:
%%time
cb_model1.fit(pool_train1,verbose=False, plot=False)
cb_model2.fit(pool_train2,verbose=False, plot=False)
cb_model3.fit(pool_train3,verbose=False, plot=False)
cb_model4.fit(pool_train4,verbose=False, plot=False)

CPU times: total: 9min 30s
Wall time: 3min 35s


In [70]:
#Получение ответов модели на тестовой выборке в локальном тестировании 
y_pred_cb1 = cb_model1.predict(pool_val).astype(int)
y_pred_cb2 = cb_model2.predict(pool_val).astype(int)
y_pred_cb3 = cb_model3.predict(pool_val).astype(int)
y_pred_cb4 = cb_model4.predict(pool_val).astype(int)
y_pred = np.hstack((y_pred_cb1, y_pred_cb2, y_pred_cb3, y_pred_cb4))

In [72]:
result = 0 
for i in range(4):
      result += recall_score(y_val[col_name[i]], y_pred[:,i], average='macro', zero_division=True)
print("Recall score",result/4)

Recall score 0.29028154482375973


In [73]:
#16 Recall score 0.29929114129558787 rs=2   0.292789
#17 Recall score 0.29028154482375973 rs=42  0.300656

### Predict

In [74]:
#Получение ответов
pool_test = Pool(test)
test_pred_1 = cb_model1.predict(pool_test)
test_pred_2 = cb_model2.predict(pool_test)
test_pred_3 = cb_model3.predict(pool_test)
test_pred_4 = cb_model4.predict(pool_test)

In [75]:
test_pred = np.hstack((test_pred_1, test_pred_2, test_pred_3, test_pred_4))
test_pred = pd.DataFrame(test_pred, columns = col_name).astype(int)

In [76]:
submission = pd.read_csv("sample_solution.csv")
submission[col_name] = test_pred
submission.head(5)

,id,Analytical thinking,Systemic thinking,Adaptability,Focus
0,10199,4,4,4,4
1,10539,4,4,4,4
2,10174,4,4,4,4
3,10465,4,4,5,4
4,10066,4,4,4,4


In [77]:
submission.to_csv('221016_18 submission cb.csv', index=None) 
submission.to_csv('submission.csv', index=None)

## RandomForestClassifier

In [78]:
# clf = RandomForestClassifier(random_state=0)
# clf.fit(X_train, y_train)

### Оценка точности

In [79]:
# y_pred = clf.predict(X_val)
# y_pred

In [80]:
# result = 0 
# for i in range(4):
#       result += recall_score(y_val[col_name[i]], y_pred[:,i], average='macro', zero_division=True)
# print("Recall score",result/4)


## Предсказание

In [81]:
# test = test.drop(columns=col_name)
# test

In [82]:
# test_pred = clf.predict(test)
# test_pred

In [83]:
# df_test_pred = pd.DataFrame(test_pred, columns = col_name).astype(int)
# df_test_pred

In [84]:
# result = pd.read_csv("sample_solution.csv")
# result[col_name] = df_test_pred
# result

In [85]:
#result.to_csv('221015_16 submission.csv', index=None) 
#result.to_csv('submission.csv', index=None)